In [20]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers




In [21]:
#Extract data from the PDF
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents

In [22]:
extracted_data = load_pdf("data/")

In [23]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [24]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 7020


In [25]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

embeddings = download_hugging_face_embeddings()

In [26]:
embeddings

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Normalize()
), model_name='sentence-transformers/all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={})

In [27]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [28]:

from pinecone import Pinecone

# Initialize Pinecone client
api_key = "76005231-c9e4-4584-8eec-1ab3080a8019"
pc = Pinecone(api_key=api_key, environment="aws", region="us-east-1", service="chatbot-frq4voi")

index_name = "chatbot"

In [39]:
def generate_embeddings(text_chunks, embeddings_model):
    embeddings_list = []
    for chunk in text_chunks:
        # Generate embedding for each chunk of text
        embedding = embeddings_model([chunk])
        embeddings_list.append(embedding[0])  # Assuming only one embedding per chunk
    return embeddings_list


In [36]:
embeddings_list = generate_embeddings(text_chunks, embeddings)

AttributeError: 'HuggingFaceEmbeddings' object has no attribute 'embed'